# Client Segmentation Clustering

We have a dataset about the stocks management of a company. We have data about orders made by different clients and also other type of transactions like shipipment orders, bank transactions stock balancing transactions, etc.

We need to focus only on the proper customer transactions and group the customers in certain clusters based on their behaviours

### Importing the dataset

We need to change the default encondig because of some charactrs in the dataset

In [47]:
import pandas as pd

df = pd.read_csv('data.csv', encoding='unicode_escape')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


#### Dataset Description:
- InvoiceNo :    Code representing each unique transaction. If this code starts with letter 'c', it indicates a cancellation.
- StockCode :   Code uniquely assigned to each distinct product.
- Description :	Description of each product.
- Quantity :	    The number of units of a product in a transaction.
- InvoiceDate :	The date and time of the transaction.
- UnitPrice :	    The unit price of the product in sterling.
- CustomerID :	Identifier uniquely assigned to each customer.
- Country :	    The country of the customer.

We want to see how many rows and columns we have

- we have 541909 rows and 8 columns

In [48]:
df.shape

(541909, 8)

We want to see what types of data we have

1. We notice that we have some columns that are normally numeric ones like InvoiceNo, StockCode that are now as objects because they also contain letters, we will need to somehow convert these to numeric type

2. We also notice that InvoiceDate is object and we will convert it to datetype

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## Data cleaning

### Handling null values:

We want to see the features with nulls

Since we want to observe the behaviour of customers we need to get rid of the data without 'Customer ID'

In [50]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

By deleting all the 'CustomerID' rows with null values it will remove also all the rows from 'Description' that have null values because the rows match

In [51]:
df.dropna(subset=['CustomerID'], inplace=True, axis=0)
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

### Checking duplicates

We have duplicates but we want to see if they are legit orders or not

In [52]:
duplicated_rows = df[df.duplicated(keep=False)]
duplicated_rows = duplicated_rows.sort_values(by=['InvoiceNo', 'StockCode'])
duplicated_rows.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,12/1/2010 11:45,1.25,17908.0,United Kingdom
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,12/1/2010 11:45,1.25,17908.0,United Kingdom
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/1/2010 11:45,4.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/1/2010 11:45,4.95,17908.0,United Kingdom
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,12/1/2010 11:45,2.10,17908.0,United Kingdom


We see that there are duplicates but it seems that the system allows buying the same product multiple times on an order

We see that on the same InvoiceNO '536381' we have the same product '71270' added with different quantities - this makes us think that it is possible to have the same product mutiple times on the same order with the same or different 'Quantity'

In [53]:
# Group by 'orderID' and 'product_code', then filter based on multiple occurrences with different quantities
duplicate_orders = df[df.duplicated(subset=['InvoiceNo', 'StockCode'], keep=False)]

print("Orders with the same product_code brought multiple times with different quantities:")
duplicate_orders.head()

Orders with the same product_code brought multiple times with different quantities:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
113,536381,71270,PHOTO CLIP LINE,1,12/1/2010 9:41,1.25,15311.0,United Kingdom
125,536381,71270,PHOTO CLIP LINE,3,12/1/2010 9:41,1.25,15311.0,United Kingdom
483,536409,90199C,5 STRAND GLASS NECKLACE CRYSTAL,3,12/1/2010 11:45,6.35,17908.0,United Kingdom
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/1/2010 11:45,4.95,17908.0,United Kingdom
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,12/1/2010 11:45,2.10,17908.0,United Kingdom


### Remove data that is not a proper product transaction 

First we need to check what StockCodes we have, we can see that we have some stock codes that are not starting with a numeric value, these are
bank charges, carriege fees, amazon fees, transport related transactions, etc. We can drop these rows because they don't help us at all 

In [54]:
non_numeric_stocks = df[~df['StockCode'].str.contains('^\\d', na=False)]
non_numeric_stocks['StockCode'].value_counts()

StockCode
POST            1196
M                465
C2               134
D                 77
DOT               16
CRUK              16
BANK CHARGES      12
PADS               4
Name: count, dtype: int64

We drop these rows

In [55]:
print(df.shape)
df.drop(index=non_numeric_stocks.index, inplace=True)
print(df.shape)

(406829, 8)
(404909, 8)


We want to see what UnitPrices we have

We notice that we have some rows with UnitPrices equal to 0 - these do not help us at all

In [56]:
df['UnitPrice'].describe()

count    404909.000000
mean          2.901129
std           4.430846
min           0.000000
25%           1.250000
50%           1.950000
75%           3.750000
max         649.500000
Name: UnitPrice, dtype: float64

We notice that we have 33 rows where UnitPrice is 0 - we can remove them

In [57]:
zero_unit_price = df[df['UnitPrice']==0]
print(zero_unit_price.value_counts().sum())

33


In [58]:
print(df.shape)
df.drop(index=zero_unit_price.index, inplace=True)
print(df.shape)

(404909, 8)
(404876, 8)


We want to see what type of orders we have, it seems we have normal orders and canceled orders. We notice that there are around 8500 canceled orders

In [59]:
invoice_codes = df['InvoiceNo']
invoice_codes.sort_values(ascending=False)

non_numeric_invoices = df[~df['InvoiceNo'].str.contains('^\\d', na=False)]
non_numeric_invoices.value_counts()


InvoiceNo  StockCode  Description                         Quantity  InvoiceDate       UnitPrice  CustomerID  Country       
C543611    82483      WOOD 2 DRAWER CABINET WHITE FINISH  -1        2/10/2011 14:38   4.95       17850.0     United Kingdom    4
C538341    22725      ALARM CLOCK BAKELIKE CHOCOLATE      -1        12/10/2010 14:03  3.75       15514.0     United Kingdom    3
           22976      CIRCUS PARADE CHILDRENS EGG CUP     -12       12/10/2010 14:03  1.25       15514.0     United Kingdom    3
           22730      ALARM CLOCK BAKELIKE IVORY          -1        12/10/2010 14:03  3.75       15514.0     United Kingdom    3
C570556    20971      PINK BLUE FELT CRAFT TRINKET BOX    -1296     10/11/2011 11:10  1.06       16029.0     United Kingdom    2
                                                                                                                              ..
C551285    82483      WOOD 2 DRAWER CABINET WHITE FINISH  -1        4/27/2011 14:07   6.95       15005

We need to handle the canceled orders in some way. It seems that they start with 'C' letter followed by an order id. What we could is to create another binary column 'CanceledOrder' and we can mark the canceled orders with 1 and the others with 0

Afterwards we need to clean the 'InvoiceNo' column by removing the 'C' letter from those invoices that have it

In [60]:
df['CanceledOrder'] = df['InvoiceNo'].str.startswith('C').astype(int)
df['InvoiceNo'] = df['InvoiceNo'].str.replace('C', '')
df['CanceledOrder'].value_counts()

CanceledOrder
0    396337
1      8539
Name: count, dtype: int64

### Removing unneeded columns

We want cluster the clients based on their behaviours so for this the relevant features would be the following:
 - Quantity
 - UnitPrice
 - InvoiceDate
 - CustomerID - not fully sure we need it
 - Country - not fully sure we need it 


We have Country column we need to see how the samples is distributed over countries

We see that the majority of the orders are from United Kingdom and they represent ~90% from the total

We can remove the 'Country' column because the dataset is unballanced in this regard

In [61]:
df['Country'].value_counts(normalize=True)
df.drop('Country', axis=1, inplace=True)
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'CanceledOrder'],
      dtype='object')

Also we can remove 'Description' column because it is not relevant

In [62]:
df.drop('Description', axis=1, inplace=True)
df.columns

Index(['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'CustomerID', 'CanceledOrder'],
      dtype='object')

We can also remove StockCode because the anlyisis is about customers not about products we sell

In [63]:
df.drop('StockCode', axis=1, inplace=True)
df.columns

Index(['InvoiceNo', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'CanceledOrder'],
      dtype='object')

### Converting columns to proper type

#### Date column
We have the 'InvoiceDate' column that is of object type, we need to convert it to datetime type and extract what we need from it. We keep only the date since the hour and minutes are not very useful for us

In [64]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.date

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 404876 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   InvoiceNo      404876 non-null  object 
 1   Quantity       404876 non-null  int64  
 2   InvoiceDate    404876 non-null  object 
 3   UnitPrice      404876 non-null  float64
 4   CustomerID     404876 non-null  float64
 5   CanceledOrder  404876 non-null  int32  
dtypes: float64(2), int32(1), int64(1), object(2)
memory usage: 20.1+ MB


#### InvoiceNo columns

We need to convert this from object to int64

In [65]:
df = df.astype({'InvoiceNo':'int64'})
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 404876 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   InvoiceNo      404876 non-null  int64  
 1   Quantity       404876 non-null  int64  
 2   InvoiceDate    404876 non-null  object 
 3   UnitPrice      404876 non-null  float64
 4   CustomerID     404876 non-null  float64
 5   CanceledOrder  404876 non-null  int32  
dtypes: float64(2), int32(1), int64(2), object(1)
memory usage: 20.1+ MB


Now all the columns are in the proper format

## Feature Engineering | Aggregating data

Since we have the same order on multiple rows we want to aggregate these to a single order, sum the Quantity for that order, the total value of the order

Also we drop the canceled orders and the 'CanceledOrder' column  -- !!!!Do we keep it or not?!!!!

In [66]:
# Calculate the total price for each invoice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Group by 'InvoiceNo'
inovice_df = df.groupby(['InvoiceNo', 'CustomerID', 'InvoiceDate', 'CanceledOrder']).agg({'Quantity': 'sum', 'TotalPrice': 'sum'}).reset_index()

#drop the canceled orders - the rows that are reperesnting canceled orders
#inovice_df = inovice_df[inovice_df['CanceledOrder'] != 1].reset_index(drop=True)

#drop the 'CaneceledOrder' column
#inovice_df.drop('CanceledOrder', axis=1, inplace=True)

inovice_df.shape

(21785, 6)

Aggregate by CustomerID

In [67]:
inovice_df.head()

,InvoiceNo,CustomerID,InvoiceDate,CanceledOrder,Quantity,TotalPrice
0,536365,17850.0,2010-12-01,0,40,139.12
1,536366,17850.0,2010-12-01,0,12,22.20
2,536367,13047.0,2010-12-01,0,83,278.73
3,536368,13047.0,2010-12-01,0,15,70.05
4,536369,13047.0,2010-12-01,0,3,17.85


I Aggregate by CustomerId, I need the number of invocies for each customer, the date of the last purchase (most recent purchase) and I sum the qunaitites of products brought and the money spent 

In [68]:
# Extract the earliest date that appears in the dataframe
earliest_date = pd.to_datetime(inovice_df['InvoiceDate'].min())
latest_date = pd.to_datetime(inovice_df['InvoiceDate'].max())


# Aggregate invoices by customer id
customer_df = inovice_df.groupby('CustomerID').agg({'InvoiceNo': 'count','InvoiceDate': ['min', 'max'], 'Quantity':'sum', 'TotalPrice':'sum', 'CanceledOrder': 'count'}).reset_index()

# Rename columns if needed
customer_df.columns = ['CustomerID', 'Total_Transactions', 'First_Purchase_Date', 'Latest_Purchase_Date', 'Total_Quantity', 'Total_Spent', 'Cancellation_Frequency']

# Convert Latest_Purchase_Date and First_Purchase_Date to datetime
customer_df['Latest_Purchase_Date'] = pd.to_datetime(customer_df['Latest_Purchase_Date'])
customer_df['First_Purchase_Date'] = pd.to_datetime(customer_df['First_Purchase_Date'])

customer_df.head()

,CustomerID,Total_Transactions,First_Purchase_Date,Latest_Purchase_Date,Total_Quantity,Total_Spent,Cancellation_Frequency
0,12346.0,2,2011-01-18,2011-01-18,0,0.00,2
1,12347.0,7,2010-12-07,2011-12-07,2458,4310.00,7
2,12348.0,4,2010-12-16,2011-09-25,2332,1437.24,4
3,12349.0,1,2011-11-21,2011-11-21,630,1457.55,1
4,12350.0,1,2011-02-02,2011-02-02,196,294.40,1


#### Recency Feature

ow I want to get the number of days passed since last purchase instead of 'LatestPurchaseDate' - but since the dataset timespan is from december 2010 until December 2011 and today we are in 2024 we need to count these days as if we are doing it from December 2011. 

- Dataset timespan: 2010-12-01 --> 2011-12-09

So we need to set a fake current date to: 2011-12-10

In [71]:
from datetime import datetime

# we set a current date variable to be the last_date in the df + 1 day -  the next day - 2011-12-10
current_date = latest_date + pd.to_timedelta(1, unit='D')

# Create a new feature 'DaysSinceLastPurchase'
customer_df['Days_Since_Last_Purchase'] = (current_date - customer_df['Latest_Purchase_Date']).dt.days
customer_df['Days_Since_First_Purchase'] = (current_date - customer_df['First_Purchase_Date']).dt.days

customer_df.head()


,CustomerID,Total_Transactions,First_Purchase_Date,Latest_Purchase_Date,Total_Quantity,Total_Spent,Cancellation_Frequency,Days_Since_Last_Purchase,Days_Since_First_Purchase
0,12346.0,2,2011-01-18,2011-01-18,0,0.00,2,326,326
1,12347.0,7,2010-12-07,2011-12-07,2458,4310.00,7,3,368
2,12348.0,4,2010-12-16,2011-09-25,2332,1437.24,4,76,359
3,12349.0,1,2011-11-21,2011-11-21,630,1457.55,1,19,19
4,12350.0,1,2011-02-02,2011-02-02,196,294.40,1,311,311


Drop the Latest_Purchase_Date and First_Purchase_Date column

In [72]:
customer_df = customer_df.drop(columns=['Latest_Purchase_Date'])
customer_df = customer_df.drop(columns=['First_Purchase_Date'])
customer_df.head()

,CustomerID,Total_Transactions,Total_Quantity,Total_Spent,Cancellation_Frequency,Days_Since_Last_Purchase,Days_Since_First_Purchase
0,12346.0,2,0,0.00,2,326,326
1,12347.0,7,2458,4310.00,7,3,368
2,12348.0,4,2332,1437.24,4,76,359
3,12349.0,1,630,1457.55,1,19,19
4,12350.0,1,196,294.40,1,311,311


#### Frequency feature

We can see how often the customer issues an order 
I will calculate the Order_Frequency by dividing the number of total transactions over the number of days since first purchase - that way I will have the order frecuency of a customer

In [84]:
# Calculate the order frequency for each customer with regard to it's first purchase - the Relative Order Frequency - this looks at the days passed
customer_df['Relative_Order_Frequency'] = customer_df['Total_Transactions'] / customer_df['Days_Since_First_Purchase']

# Calculate the order frequency for each customer with regard to all the dataset - to all the period captured in the dataset
# the Absolute Order Frequency - this looks at what is the wight of the transactions made by the customer devied by the total number of transactions
customer_df['Absolute_Order_Frequency'] = customer_df['Total_Transactions'] / customer_df['Total_Transactions'].sum()

customer_df.sort_values(by='Absolute_Order_Frequency', ascending=False).head()

,CustomerID,Total_Transactions,Total_Quantity,Total_Spent,Cancellation_Frequency,Days_Since_Last_Purchase,Days_Since_First_Purchase,Relative_Order_Frequency,Absolute_Order_Frequency
1891,14911.0,242,76848,128882.13,242,2,374,0.647059,0.011109
328,12748.0,217,23976,30486.53,217,1,374,0.580214,0.009961
4033,17841.0,169,22850,40333.22,169,2,374,0.451872,0.007758
1670,14606.0,125,5969,11567.10,125,2,374,0.334225,0.005738
566,13089.0,118,30787,57385.88,118,3,370,0.318919,0.005417


Now I can drop the column 'Days_Since_First_Purchase'

In [26]:
customer_df = customer_df.drop(columns='Days_Since_First_Purchase')
customer_df.head()

,CustomerID,Total_Transactions,Total_Quantity,Total_Spent,Cancellation_Frequency,Days_Since_Last_Purchase,Order_Frequency
0,12346.0,2,0,0.00,2,326,0.006135
1,12347.0,7,2458,4310.00,7,3,0.019022
2,12348.0,4,2332,1437.24,4,76,0.011142
3,12349.0,1,630,1457.55,1,19,0.052632
4,12350.0,1,196,294.40,1,311,0.003215


#### Monetary features

- We have the Total_Spent that gives us the total number of money a customer has spent 
- We might need the average money spent by a customer per transaction


In [27]:
customer_df['Average_Transaction_Value'] = customer_df['Total_Spent'] / customer_df['Total_Transactions']
customer_df.head()

,CustomerID,Total_Transactions,Total_Quantity,Total_Spent,Cancellation_Frequency,Days_Since_Last_Purchase,Order_Frequency,Average_Transaction_Value
0,12346.0,2,0,0.00,2,326,0.006135,0.000000
1,12347.0,7,2458,4310.00,7,3,0.019022,615.714286
2,12348.0,4,2332,1437.24,4,76,0.011142,359.310000
3,12349.0,1,630,1457.55,1,19,0.052632,1457.550000
4,12350.0,1,196,294.40,1,311,0.003215,294.400000


Now we can see the customers that don't bring any value

We have a total of 40 customers that don't bring any value. We need to see what we do with them

In [29]:
no_value_customers = customer_df[customer_df['Total_Spent'] <= 0 ]
no_value_customers.value_counts().sum()

40

We need the following features: - these need to be done by aggregating the customers
- Number of days since last order : this gives us rencency of pruchase
- Average number of days between orders ???
- Number of orders / customer
- Average spending per order of a customer